# 原價屋 零件清單 Parser

In [4]:
#!/usr/bin/python3

import re
import requests
from functools import reduce
from bs4 import BeautifulSoup
from tabulate import tabulate

def parsePage(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:91.0) Gecko/20100101 Firefox/91.0"
    }
    # Send an HTTP request to the URL with custom headers
    response = requests.get(url, headers=headers)
    # Check if the request was successful (status code 200)
    assert(response.status_code == 200)
    return BeautifulSoup(response.text, 'html.parser')

In [5]:
def processAnyPage(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        data = []
        for div in div_tags:
            data += [div.text]
        allData += [data]
    #print(tabulate(allData, tablefmt="grid"))
    print(tabulate(sorted(allData), tablefmt="plain"))
processAnyPage(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=12"))

ACER 宏�� Predator Arc A770 16G(2400MHz/26.7cm/三年)Predator冷卻系統           繪圖核心：Arc A770                 核心時脈：2400MHz               輸出介面：1*HDMI(2.1) / 3*DP(2.0)  電源接口：1*8pin + 1*8pin                      顯卡長度：26.7公分          商品保固：三年保 / ACER 客服  含稅：NT11590  ♦開箱討論  Buy
ACER 宏�� Predator Arc A770 16G(2400MHz/26.7cm/三年)Predator冷卻系統 *任搭價   繪圖核心：Arc A770                 核心時脈：2400MHz               輸出介面：1*HDMI(2.1) / 3*DP(2.0)  電源接口：1*8pin + 1*8pin                      顯卡長度：26.7公分          商品保固：三年保 / ACER 客服  含稅：NT9990  ♦開箱討論  Buy   加送 Acer Predator Frostblade 120 ARGB風扇二顆 (市價 NT.$390)
AMD Radeon Pro W5700(8GB GDDR6 256bit/SP:2304/三年保固) 工作站級繪圖卡         繪圖核心：AMD RDNA                 SP 數：2304                     輸出介面：5*miniDP(1.4)/USB-C      電源接口：1*6pin + 1*8pin                      顯卡長度：26.7公分          商品保固：三年保              含稅：NT19500  ♦開箱討論  Buy
AMD Radeon Pro W6400 (4GB GDDR6 64bit/SP:768/三年保固) 工作站級繪圖卡          繪圖核心：AMD RDNA 2               SP 數：768                      輸出介面：2*DP(1.4)          

In [6]:
def processCasePage(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title, size, price  = "", "", ""
        minLen = volume = 0.0
        for div in div_tags:
            #print(div)
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
            else:
                m = re.search("尺寸：", div.text)
                if not m: continue
                size = div.text.replace("/", "*")
                if "~" in size: continue
                m = re.search("(\d+\.?\d*)\*(\d+\.?\d*)\*(\d+\.?\d*)", size)
                if m:
                    #print(size, m.groups())
                    szSpec = list(map(float, m.groups()))
                    volume = reduce(lambda x, y: x*y, szSpec)
                    minLen = min(szSpec)
                else:
                    print(f"CANT PARSE: {size}")
        allData += [(minLen, volume, price, size, title)]

    print(tabulate(sorted(allData), tablefmt="plain"))


processCasePage(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=14"))

 0         0     45000  尺寸：100*68.9~117.5*74                          聯力 DK-04F 電動可調式電腦桌/顯卡長39/CPU高18.5/E-ATX【訂】
 0         0      5190  尺寸：37.1*17.3*21.1~27.2                        Phanteks Enthoo Evolv Shift XT 銀 顯卡32.4/U高7.2/ITX(ES121XT_DGS01)【SFX】
 0         0      5190  尺寸：37.1*17.3*21.1~27.2                        Phanteks Enthoo Evolv Shift XT 銀 顯卡32.4/U高7.2/ITX(ES121XT_DGS01)【SFX】
 0         0      5190  尺寸：37.1*17.3*21.1~27.2                        Phanteks Enthoo Evolv Shift XT 黑 顯卡32.4/U高7.2/ITX(ES121XT_DBK01)【SFX】
 0         0      5190  尺寸：37.1*17.3*21.1~27.2                        Phanteks Enthoo Evolv Shift XT 黑 顯卡32.4/U高7.2/ITX(ES121XT_DBK01)【SFX】
 0         0     60000  尺寸：140*68.9~117.5*78                          聯力 DK-05F 電動可調式電腦桌/顯卡長40/CPU高18.5/E-ATX【訂】
 8     11248      2260  尺寸：8*38*37                                    銀欣 小烏鴉2 顯卡長33/CPU高5.8/SSD*2/ITX(RVZ02B) 【SFX】
 8     11248      2260  尺寸：8*38*37                                    銀欣 小烏鴉2 顯卡長33/CPU高5.8/SSD*2/

In [7]:
def processCpuPage(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        for div in div_tags:
            #print(div)
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        if re.search("Intel", title, re.IGNORECASE) and "-13" in title:
            allData += [(float(price), title)]
    print(tabulate(sorted(allData), tablefmt="plain"))
processCpuPage(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=4"))

 3650  Intel i3-13100F【4核/8緒】3.4GHz(↑4.5GHz)/12M/無內顯/58W【代理盒】
 4450  Intel i3-13100【4核/8緒】3.4GHz(↑4.5GHz)/12M/UHD730/60W【代理盒】
 6700  Intel i5-13400F【10核/16緒】2.5GHz(↑4.6G)/20M/無內顯/65W【代理盒裝】
 7300  Intel i5-13400【10核/16緒】2.5GHz(↑4.6G)/20M/UHD730/65W【代理盒裝】
 7900  Intel i5-13500【14核/20緒】2.5GHz(↑4.8G)/24M/UHD770/65W【代理盒裝】
12300  Intel i7-13700F【16核/24緒】2.1GHz(↑5.2G)/30M/無內顯/65W【代理盒裝】
12800  Intel i7-13700【16核/24緒】2.1GHz(↑5.2G)/30M/UHD770/65W【代理盒裝】
18000  Intel i9-13900F【24核/32緒】2.0GHz(↑5.6G)/36M/無內顯/65W【代理盒裝】
18800  Intel i9-13900【24核/32緒】2.0GHz(↑5.6G)/36M/UHD770/65W【代理盒裝】
24000  Intel i9-13900KS【24核/32緒】3.2G(↑6.0G)/36M/UHD770/無風扇/150W【代理盒裝】


In [8]:
def processGpuPage(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        for div in div_tags:
            #print(div)
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        if "3060" in title:
            allData += [(float(price), title)]

    print(tabulate(sorted(allData), tablefmt="plain"))
processGpuPage(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=12"))

 9990  ZOTAC RTX3060 TWIN EDGE(1777MHz/22.4cm/雙風扇/註四年)*送原廠大鼠墊
 9990  華碩 PH-RTX3060-12G-V2(1807MHz/17.7cm/註冊四年保) 優惠至4/30 !
10488  INNO3D RTX3060 TWIN X2 OC LHR(1792MHz/24cm/雙風扇/註冊四年保)
10490  ZOTAC RTX3060 TWIN EDGE OC(1807MHz/22.4cm/雙風扇/)*優惠到4/30
10490  華碩 DUAL-RTX3060-O8G(1867MHz/20cm/註冊四年) 軸向式雙風扇
10790  華碩 DUAL-RTX3060-O8G-WHITE(1867MHz/20cm) 雪原豹回歸.雙滾珠軸承風扇
10990  微星 RTX3060 VENTUS 2X 12G OC(1807MHz/23.5cm/註四年) 優惠至4/30 !
10990  技嘉 RTX3060 EAGLE OC 12G(rev2.0)(1807MHz/24.2cm/註五年)
10990  技嘉 RTX3060 GAMING OC 8G(1807MHz/19.8cm/註五年)刀鋒扇葉正逆轉
10990  華碩 DUAL-RTX3060-O12G-V2(OC上1867MHz/20cm/註四年) 優惠至4/30 !
11488  INNO3D RTX3060 iChill X3 RED(1837MHz/30cm/註五年/三風扇/註冊五年保)
11888  INNO3D RTX3060Ti TWIN X2 LHR(1665MHz/24cm/註冊四年保)*缺貨免費升級OC版
12290  華碩 DUAL-RTX3060-O12G-WHITE(1867MHz/20cm/註四年/白)雪原豹回歸.雙滾珠軸承
12488  INNO3D RTX3060Ti TWIN X2 OC LHR(1680MHz/24cm/註冊四年保)
12490  技嘉 RTX3060 GAMING OC 12G(rev2.0)(1837MHz/28.2cm/註冊五年)
12790  ZOTAC RTX3060Ti GDDR6X Twin Edge(1665MHz/23.19cm)*3070同級散熱+送大鼠墊
12890  ZOT

In [9]:
def processMotherBoardPage(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = size = cpu = ram = display = network = ""
        for div in div_tags:
            #print(div)
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
            elif "尺寸" in div.text:
                size = div.text
            elif "CPU：" in div.text:
                cpu = div.text
            elif "記憶體：" in div.text:
                ram = div.text
            elif "顯示：" in div.text:
                display = div.text
            elif "網路：" in div.text:
                network = div.text
        if re.search("ITX", size, re.IGNORECASE) \
            and re.search("1700", cpu, re.IGNORECASE) \
            and re.search("5", display, re.IGNORECASE): #PCIE 5.0
            allData += [(float(price), title, cpu, ram, size, display, network)]

    print(tabulate(sorted(allData), tablefmt="plain"))
processMotherBoardPage(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=5"))


 6190  微星 B760I EDGE WIFI DDR4(Mini-ITX/Realtek 2.5Gb+Wi-Fi 6E/註五年)8+1+1相供電    CPU：LGA 1700 for 12th/13th Gen Intel    尺寸：Mini ITX(DDR4) / 8層板  顯示：1*HDMI(2.1) / 1*DP(1.4) / PCIe 5.0        網路：Realtek 2.5Gb + Intel Wi-Fi 6E + BT5.3
 6590  華碩 ROG STRIX B760-I GAMING WIFI(Mini ITX/Intel 2.5Gb+Wi-Fi 6E/註五年)8+1相    CPU：LGA 1700 for 12th/13th Gen Intel    尺寸：Mini ITX(DDR5)          顯示：1*HDMI(2.1) / 1*DP(1.4) / PCIe 5.0        網路：Intel 2.5Gb + Wi-Fi 6E + BT5.3
 9000  華擎 Z790M-ITX WiFi(Mini-ITX/Intel 2.5Gb+Wi-Fi 6E/註冊五年)7+1+1相電源.支援eDP  CPU：LGA 1700 for 12th/13th Gen Intel    尺寸：Mini ITX(DDR5)          顯示：1*HDMI(2.1) / 1*DP(1.4) / PCIe 5.0        網路：Intel 2.5Gb + Wi-Fi 6E + BT
11590  微星 Z790I EDGE WIFI(Mini-ITX/Intel 2.5Gb+Wi-Fi 6E/註冊五年)10+1+1相電源        CPU：LGA 1700 for 12th/13th Gen Intel    尺寸：Mini ITX(DDR5)          顯示：1*HDMI(2.1) / 1*DP(1.4) / PCIe 5.0        網路：Intel 2.5Gb + Intel Wi-Fi 6E + BT5.3
13990  華碩 ROG STRIX Z790-I GAMING WIFI(Mini-ITX/Intel 2.5Gb+Wi-Fi 6E)10+1相電源      CPU：LGA 170

In [10]:
def processPsu(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        price = power = 0
        for div in div_tags:
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
                m = re.search("(\d\d\d)[Ww]", title)
                if m:
                    power = int(m.group(1))
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        if "全模" in title and "SFX規格" in title and "金" in title\
            and power >= 600:
            allData += [(float(price), power, title)]

    print(tabulate(sorted(allData), tablefmt="plain"))
processPsu(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=15"))


2990  650  Montech 創世紀 CENTURY MINI 650W 雙8/金牌/全模組/全日系/7年保【SFX規格】
2990  650  Montech 創世紀 CENTURY MINI 650W 雙8/金牌/全模組/全日系/7年保【SFX規格】
3290  650  全漢 金鋼彈 650W 雙8/金牌/全模/全日系/DC-DC/10年保固【SFX規格】
3290  650  全漢 金鋼彈 650W/金牌/全模/全日系/DC-DC/10年保【SFX規格】
3290  750  Apexgaming(首利) SFX-750W 雙8/白金/全模組/全日系/鋁合金外殼/10年保【SFX規格】
3290  750  Apexgaming(首利) SFX-750W 雙8/白金/全模組/全日系/鋁合金外殼/10年保【SFX規格】
3290  750  Montech 創世紀 CENTURY MINI 750W 雙8/金牌/全模組/全日系/7年保【SFX規格】
3290  750  Montech 創世紀 CENTURY MINI 750W 雙8/金牌/全模組/全日系/7年保【SFX規格】
3590  750  全漢 金鋼彈 750W 雙8/金牌/全模/全日系/DC-DC/10年保【SFX規格】
3590  750  全漢 金鋼彈 750W 雙8/金牌/全模/全日系/DC-DC/10年保固【SFX規格】
3690  850  Apexgaming(首利) SFX-850W 白色/雙8/白金/全模/全日系/鋁合金外殼/10年【SFX規格】
3690  850  Apexgaming(首利) SFX-850W 白色/雙8/白金/全模/全日系/鋁合金外殼/10年【SFX規格】
3740  650  Phanteks Revolt SFX 650W 雙8/金牌/全模組/10年保(PH-P650GSF)【SFX規格】
3740  650  Phanteks Revolt SFX 650W 雙8/金牌/全模組/10年保(PH-P650GSF)【SFX規格】
3790  650  酷碼 V650 SFX GOLD 650W 雙8/金牌/全模組/10年保【SFX規格】
3790  650  酷碼 V650 SFX GOLD 650W 雙8/金牌/全模組/10年保【SFX規格】
3

In [11]:
def processCooler(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        height = price = 0
        for div in div_tags:
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
                m = re.search("/(高.*?)/", title)
                if m:
                  height = re.search("(\d+\.?\d?)", m.group(1)).group(1)
                  #print(m.group(1), height)
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        #if (height != 0.0):
        if "導熱係數" in title:
          allData += [(float(height), float(price), title)]
    print(tabulate(sorted(allData), tablefmt="plain"))
processCooler(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=10"))


0   70  酷碼 HTK-002 美國道康膏/導熱係數 0.8W/m-K
0   99  酷碼 IC Value V1 超值型散熱膏/導熱係數 1.85W/mK(RG-ICV1-TW20-R1)
0  109  Montech AP3 散熱膏/4公克/附刮刀/導熱係數 5.99W/mK
0  149  酷碼 MasterGel Pro V2 長效散熱膏/導熱係數 9W/mK(MGY-ZOSG-N15M-R3)
0  169  快睿 CRYORIG CP15 導熱膏/4公克/附塗抹卡片.清潔片/導熱係數 5.7W/mK
0  190  Scythe Thermal Elixer 2 導熱膏/5g/導熱係數 3.5W/mK(SCTE-2000)
0  190  利民 Thermalright TF9 散熱膏/1.5公克/導熱係數 14W/mK
0  199  酷碼 IceFusion 新酷媽涼膏 V2/40公克/導熱係數 5 W/mK(RG-ICF-CWR3-GP)
0  249  酷碼 CryoFuze CF-14 超效散熱膏/導熱係數 14W/mK(MGZ-NDSG-N07M-R2)
0  250  利民 Thermalright TF8 散熱膏/2公克/導熱係數 13.8W/mK
0  280  利民 Thermalright Silver King 液態金屬膏/1公克/導熱係數 79W/mK【避免接觸鋁製品】
0  299  酷碼 MasterGel Pro 長效散熱膏/導熱係數 8W/mK/扁平開口針筒(MGY-ZOSG-N15M-R2)
0  360  利民 Thermalright TFX 散熱膏/2公克/導熱係數 14.3W/mK
0  590  利民 Thermalright TF8 散熱膏/5.8公克/導熱係數 13.8W/mK


In [46]:
def processWaterCooler(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        height = price = 0
        for div in div_tags:
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
                m = re.search("/(厚.*?)/", title)
                if not m:
                    m = re.search("/(厚.*?\d.*)\W", title)
                if m:
                    #print(m.group(1))
                    height = re.search("(\d+\.?\d?)", m.group(1)).group(1)
                  #print(m.group(1), height)
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        #if "360" not in title:
        if "" in title:
            allData += [(float(price), float(height), title)]
            #allData += [(float(height), float(price), title)]

    print(tabulate(sorted(allData), tablefmt="plain"))
processWaterCooler(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=11"))


  100  0    酷碼 CM LGA1700 水冷專用扣具(603005870-GP)
  790  0    曜越 T1000 透明水冷液/1000ml(CL-W245)【共7種顏色請詢問】
 1490  5.2  酷碼 MasterLiquid Lite 120 水冷/厚:5.2cm/2年/MLW-D12M-A20PW-R1【WXH】
 1990  5.2  喬思伯 TG-240 白色版 /A.RGB/隱藏式理線/可旋轉LOGO/厚:5.2cm/3年【WXHZ】
 1990  5.2  喬思伯 TG-240 黑色版 /A.RGB/隱藏式理線/可旋轉LOGO/厚:5.2cm/3年【WXHZ】
 2290  5.2  海盜船 iCUE H100i RGB PRO XT 240/RGB冷頭/厚:5.2/5年(9060043)【WXHYZ】▼下殺到 4/30 21:00
 2290  5.2  華碩 TUF Gaming LC 120 ARGB 水冷 /厚:5.2cm/Aura Sync/6年【WXHZ】
 2290  5.2  酷碼 MasterLiquid ML240L V2 RGB水冷/厚:5.2/2年/D24M-A18PC-R2【WXHZ】
 2590  0    保銳 安耐美 LIQMAX III ARGB 虹彩晶凌 240(LMT240-ARGB)【WXZ】
 2590  5.2  Apexgaming TITAN 泰坦360 水冷散熱器/A.RGB冷頭+風扇/厚:5.2/3年保【WXZ】
 2590  5.2  微星 MAG CoreLiquid P240 /鏡面烤漆冷頭/厚:5.2(註冊3+2年)【WXHZ】
 2690  0    保銳 安耐美 LIQMAX III ARGB 虹彩晶凌 240 雪白版(LMT240-W-ARGB)【WXZ】
 2690  0    銀欣 PF240 ARGB 水冷散熱器/A.RGB冷頭+風扇/2年保(SST-PF240-ARGB)【WXHZ】
 2690  5.2  Antec Symphony 240 ARGB 水冷/琴鍵感鏡面冷頭/厚:5.2cm/3年【WXHZ】
 2690  5.2  COUGAR AQUA ARGB 240 水冷/A.RGB冷頭+風扇/厚:5.2cm/附遙控器/5年保【WXHZ】
 26

In [41]:
def processSSD(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        readS = writeS = price = 0
        for div in div_tags:
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
                m = re.search("讀:?(\d+)", title)
                if m:
                    readS = int(float(m.group(1)))
                m = re.search("寫:?(\d+)", title)
                if m:
                    writeS = int(float(m.group(1)))
                if not readS or not writeS:
                    print(title)
            elif div.has_attr('class') and 'x' in div['class']:
                price = re.search("NT(\d+)\W", div.text).group(1)
        if "2T" in title and readS >= 6000:#re.search("M.2", title, re.IGNORECASE):
            allData += [(readS, writeS, float(price), title)]
    print(tabulate(sorted(allData), tablefmt="plain"))
processSSD(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=7"))


Seagate FireCuda 530 1TB(散熱片)曼達洛人-貝斯卡鋼限量版~加送seagate滑鼠墊再送撲克牌
6600  5500  4350  美光Micron Crucial P5 Plus 2TB/M.2 PCIe 2280/讀:6600M/寫:5500M/3D TLC顆粒/五年▼下殺到 4/16 23:59
7000  5100  4799  三星 Samsung 980 PRO 2TB NVMe Gen4/M.2 PCIe 讀:7000/寫:5100(五年)▼下殺到 4/16 23:59
7000  5100  5299  三星 Samsung 980 PRO 2TB含散熱片 NVMe Gen4/M.2 PCIe 讀:7000/寫:5100(五年)▼下殺到 4/16 23:59
7000  6500  5300  Solidigm P44 Pro 2TB/Gen4 讀:7000M/寫:6500M/ DRAM快取/五年
7000  6850  3999  UMAX M1500 2TB/Gen4 PCIe*4 2280/讀:7000M/寫:6850M【五年】奈米碳鍍銅箔散熱片
7000  6850  4490  技嘉 AORUS NVMe Gen4 7000S Prem 2TB 散熱防護 讀:7000/寫:6850/TLC【五年保】
7000  6850  5099  KLEVV CRAS C920 2TB/Gen4 PCIe*4/讀:7000M/寫:6850M/TLC【五年】DRAM 快取
7000  6900  5299  十銓 T-ForceA440黑曜女神(雙散熱)2TB/Gen4 讀:7000M/寫:6900M(五年)
7000  7000  4999  金士頓 KC3000 2TB/Gen4 PCIe*4/讀:7000/寫:7000/TLC~加送LED三合一充電線~
7300  6600  5990  WD 黑標 SN850X 2TB/Gen4 PCIe*4/讀:7300/寫:6600/TLC/電競級【五年保】
7300  6600  6780  WD 黑標 SN850X 2TB/含散熱片Gen4 PCIe*4/讀:7300/寫:6600/TLC/電競級【五年保】
7300  6900  7150  Seagate FireCuda 53

In [33]:
def processRAM(soup):
    span_tags = soup.find_all('span')

    allData = []
    for span in span_tags:
        div_tags = span.find_all('div')
        title = ""
        cl = gen = freq = price = 0
        for div in div_tags:
            if div.has_attr('class') and 't' in div['class']:
                title = div.text
                m = re.search("\WDD?R?(\d)", title)
                if m:
                    gen = int(float(m.group(1)))
                m = re.search("(\d\d\d\d)M?", title)
                if m:
                    freq = int(float(m.group(1)))
                m = re.search("CL(\d\d)", title)
                if m:
                    cl = int(float(m.group(1)))

                if not freq or not gen:
                    if "DDR3" not in title:
                        print(f"check gen/freq regex: {title}")
            elif div.has_attr('class') and 'x' in div['class']:
                price = float(re.search("NT(\d+)\W", div.text).group(1))
        if "32G" in title and gen == 5 and freq == 5600 and "雙通16G" not in title:
            allData += [(freq, cl, price, title)]
    print(tabulate(sorted(allData), tablefmt="plain"))
processRAM(parsePage("https://www.coolpc.com.tw/eachview.php?IGrp=6"))


5600  36  6788  金士頓 64GB(雙通32GB*2) DDR5-5600/CL36 FURY Beast Black白 (獸獵者)【具雙參數】
5600  36  6788  金士頓 64GB(雙通32GB*2) DDR5-5600/CL36 FURY Beast Black黑 (獸獵者)【具雙參數】
5600  36  8990  芝奇G.SKILL Ripjaws S5焰刃 雙通32G*2 D5-5600/CL36白( F5-5600J3636D32GX2-RS5W)
5600  36  8990  芝奇G.SKILL Ripjaws S5焰刃 雙通32G*2 D5-5600/CL36黑( F5-5600J3636D32GX2-RS5K)
5600  36  8990  芝奇G.SKILL 幻鋒戟RGB 雙通32G*2 D5-5600/CL36黑( F5-5600J3636D32GX2-TZ5RK)
5600  40  3550  金士頓 32GB DDR5-5600/CL40 FURY Beast(獸獵者)(KF556C40BB-32)
5600  40  3800  金士頓 32GB DDR5-5600/CL40 FURY Beast RGB(獸獵者)(KF556C40BBA-32)
5600  46  2999  威剛 32GB DDR5-5600/CL46 (AD5U560032G-S)
5600  46  2999  威剛 NB 32GB DDR5-5600/CL46 (AD5S560032G-S)
5600  46  3399  美光Micron Crucial 單條32GB DDR5 5600/CL46(原生顆粒製)【具XMP、EXPO參數】
